In [1]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import MessagePassing #MessagePassing jest używana do definiowania warstw przetwarzania wiadomości w grafach
from torch_geometric.utils import add_self_loops, softmax

c:\Users\olame\miniconda3\envs\gnn\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
class GATLayer(MessagePassing):
    def __init__(self, in_channels, out_channels, heads=1, concat=True, negative_slope=0.2, dropout=0, **kwargs):
        super(GATLayer, self).__init__(aggr='add', **kwargs)  
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.heads = heads
        self.concat = concat
        self.negative_slope = negative_slope
        self.dropout = dropout

        self.weight = torch.nn.Parameter(torch.Tensor(in_channels, heads * out_channels))
        self.att = torch.nn.Parameter(torch.Tensor(1, heads, 2 * out_channels))

        self.reset_parameters()

    def reset_parameters(self):
        torch.nn.init.xavier_uniform_(self.weight)
        torch.nn.init.xavier_uniform_(self.att)

    def forward(self, x, edge_index):
        # Step 1: Add self-loops to the adjacency matrix
        edge_index, _ = add_self_loops(edge_index, num_nodes=x.size(0))

        # Step 2: Linearly transform node feature matrix with weight matrix
        x = torch.matmul(x, self.weight)
        
        # Step 3: Start propagating messages
        return self.propagate(edge_index, x=x)

    def message_and_aggregate(self, adj_t, x):
       
        x_i = x[adj_t.row]  # Source node features
        x_j = x[adj_t.col]  # Target node features
        
        # Calculate attention scores
        alpha = torch.cat([x_i, x_j], dim=-1)
        alpha = F.leaky_relu(torch.matmul(alpha, self.att), self.negative_slope)
        alpha = softmax(alpha, adj_t.col)  # Normalize attention scores
        alpha = F.dropout(alpha, p=self.dropout, training=self.training)

        # Perform weighted aggregation
        out = alpha * x_i  # Weight source node features by attention scores
        out = torch.sparse.sum(adj_t, dim=0)  # Aggregate messages by summation
        
        return out

    def __repr__(self):
        return '{}(in_channels={}, out_channels={}, heads={})'.format(self.__class__.__name__, self.in_channels, self.out_channels, self.heads)




In [3]:
in_channels = 16
out_channels = 8
heads = 4

gat_layer = GATLayer(in_channels, out_channels, heads=heads)
print(gat_layer)

GATLayer(in_channels=16, out_channels=8, heads=4)


#   COO CSR

In [4]:
import torch
import torch.nn.functional as F
import torch.optim as optim
import torch_geometric
from torch_geometric.data import Data
from torch_geometric.utils import to_scipy_sparse_matrix, add_self_loops, softmax
from scipy.sparse import coo_matrix, csr_matrix
import numpy as np
import time

In [5]:
def measure_time(func):
    start = time.time()
    func()
    end = time.time()
    return end - start

In [6]:
class GATModel(torch.nn.Module):
    def __init__(self, in_channels, out_channels, heads=1):
        super(GATModel, self).__init__()
        self.gat_layer = GATLayer(in_channels, out_channels, heads)

    def forward(self, x, edge_index):
        return self.gat_layer(x, edge_index)

In [20]:
def generate_graph_data(num_nodes, num_features, num_classes, edge_density):
    x = torch.rand((num_nodes, num_features), dtype=torch.float32)
    y = torch.randint(0, num_classes, (num_nodes,), dtype=torch.long)
    edge_index = torch_geometric.utils.dense_to_sparse(torch.bernoulli(torch.full((num_nodes, num_nodes), edge_density)))[0]
    return Data(x=x, edge_index=edge_index, y=y)

In [8]:
def train_epoch(model, data, optimizer, criterion, device):
    model.train()
    data = data.to(device)
    optimizer.zero_grad()
    output = model(data.x, data.edge_index)
    loss = criterion(output, data.y)
    loss.backward()
    optimizer.step()

def inference(model, data, device):
    model.eval()
    data = data.to(device)
    with torch.no_grad():
        output = model(data.x, data.edge_index)
    return output

In [9]:
num_nodes = 1000
num_features = 100
num_classes = 10
edge_density = 0.01

In [10]:
data = generate_graph_data(num_nodes, num_features, num_classes, edge_density)

In [11]:
adj_coo = to_scipy_sparse_matrix(data.edge_index).tocoo()
adj_csr = adj_coo.tocsr()


In [12]:
coo_tensor = torch.sparse_coo_tensor([adj_coo.row, adj_coo.col], adj_coo.data, size=adj_coo.shape)
data_coo = Data(x=data.x, edge_index=coo_tensor._indices(), y=data.y)

C:\Users\olame\AppData\Local\Temp\ipykernel_22024\1569400792.py:1: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  C:\cb\pytorch_1000000000000\work\torch\csrc\utils\tensor_new.cpp:204.)
  coo_tensor = torch.sparse_coo_tensor([adj_coo.row, adj_coo.col], adj_coo.data, size=adj_coo.shape)


In [13]:
csr_tensor = torch.sparse_csr_tensor(crow_indices=adj_csr.indptr, col_indices=adj_csr.indices, values=adj_csr.data, size=adj_csr.shape)
#data_csr = Data(x=data.x, edge_index=csr_tensor._indices(), y=data.y)

C:\Users\olame\AppData\Local\Temp\ipykernel_22024\3046264546.py:1: UserWarning: Sparse CSR tensor support is in beta state. If you miss a functionality in the sparse tensor support, please submit a feature request to https://github.com/pytorch/pytorch/issues. (Triggered internally at  C:\cb\pytorch_1000000000000\work\aten\src\ATen\SparseCsrTensorImpl.cpp:68.)
  csr_tensor = torch.sparse_csr_tensor(crow_indices=adj_csr.indptr, col_indices=adj_csr.indices, values=adj_csr.data, size=adj_csr.shape)


In [14]:
device_cpu = torch.device('cpu')
device_gpu = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [15]:
model = GATModel(num_features, num_classes, heads=4)
optimizer = optim.Adam(model.parameters(), lr=0.01)
criterion = torch.nn.CrossEntropyLoss()

In [16]:
# Pomiar czasu treningu i inferencji dla COO na CPU
time_coo_cpu_train = measure_time(lambda: train_epoch(model, data, optimizer, criterion, device_cpu))
time_coo_cpu_infer = measure_time(lambda: inference(model, data, device_cpu))

# Pomiar czasu treningu i inferencji dla CSR na CPU
#data_csr = Data(x=data.x, edge_index=csr_tensor._indices(), y=data.y)
#time_csr_cpu_train = measure_time(lambda: train_epoch(model, data_csr, optimizer, criterion, device_cpu))
#time_csr_cpu_infer = measure_time(lambda: inference(model, data_csr, device_cpu))

# Pomiar czasu treningu i inferencji dla COO na GPU
time_coo_gpu_train = measure_time(lambda: train_epoch(model, data, optimizer, criterion, device_gpu))
time_coo_gpu_infer = measure_time(lambda: inference(model, data, device_gpu))

# Pomiar czasu treningu i inferencji dla CSR na GPU
#time_csr_gpu_train = measure_time(lambda: train_epoch(model, data_csr, optimizer, criterion, device_gpu))
#time_csr_gpu_infer = measure_time(lambda: inference(model, data_csr, device_gpu))

In [18]:
print("COO CPU Train Time: {:.4f} seconds".format(time_coo_cpu_train))
print("COO CPU Inference Time: {:.4f} seconds".format(time_coo_cpu_infer))
#print("CSR CPU Train Time: {:.4f} seconds".format(time_csr_cpu_train))
#print("CSR CPU Inference Time: {:.4f} seconds".format(time_csr_cpu_infer))
print("COO GPU Train Time: {:.4f} seconds".format(time_coo_gpu_train))
print("COO GPU Inference Time: {:.4f} seconds".format(time_coo_gpu_infer))
#print("CSR GPU Train Time: {:.4f} seconds".format(time_csr_gpu_train))
#print("CSR GPU Inference Time: {:.4f} seconds".format(time_csr_gpu_infer))


COO CPU Train Time: 0.0141 seconds
COO CPU Inference Time: 0.0030 seconds
COO GPU Train Time: 0.0078 seconds
COO GPU Inference Time: 0.0030 seconds
